Specific game:
- https://api.rawg.io/api/games/rimworld
- https://api.rawg.io/api/games/grand-theft-auto-v
- https://rawg.io/games/grand-theft-auto-v

## Get info of each game

In [1]:
import csv
import requests
import json
from pprint import pprint
from time import time
import concurrent.futures
import functools

In [2]:
csv_data = []
with open("data/game_id.csv", "r") as f:
    csv_data = list(csv.reader(f))
for i, val in enumerate(csv_data):
    print(val)
    if i==10: break

['3498', 'grand-theft-auto-v']
['4200', 'portal-2']
['3328', 'the-witcher-3-wild-hunt']
['12020', 'left-4-dead-2']
['5679', 'the-elder-scrolls-v-skyrim']
['802', 'borderlands-2']
['13536', 'portal']
['3439', 'life-is-strange-episode-1-2']
['4291', 'counter-strike-global-offensive']
['4062', 'bioshock-infinite']
['1030', 'limbo']


In [10]:
def threading(start_index, games_per_worker, urls):
    for url in urls[start_index : start_index + games_per_worker]:
        try:
            # Get json reqquest
            json_data = json.loads(requests.get(url, headers=headers).text)

            # Only include wanted keys
            D = {k:v for k,v in json_data.items() if k in include}

            # Clean up dictionary
            D["platforms"] = []
            for platform in json_data["platforms"]:
                D["platforms"].append(platform["platform"]["name"])

            for key in ("developers", "genres", "publishers"):
                D[key] = []
                for data in json_data[key]:
                    D[key].append(data["name"])

            if json_data["esrb_rating"]:
                D["esrb_rating"] = json_data["esrb_rating"]["name"]

            name = D["id"]
            with open(f"data/game_info/{name}.json","w", encoding="utf-8") as f:
                json.dump(D, f)
        except:
            print(f"Error with {url}")

## Multithreading

#### Preparation

In [12]:
headers = { 'User-Agent': 'App Name: Education purpose',}
include = {"id",
           "slug",
           "name",
           "metacritic",
           "released",
           "tba",
           "updated",
           "website",
           "rating",
           "rating_top",
           "added_by_status",
           "playtime",
           "achievements_count",
           "ratings_count",
           "suggestions_count",
           "game_series_count",
           "reviews_count",
           "platforms",
           "developers",
           "genres",
           "publishers",
           "esrb_rating",
           }

max_workers = 100
start_game_index = 0 # min: 0
end_game_index = 100 # max: 345707
number_of_games = end_game_index - start_game_index
games_per_worker = int(number_of_games/max_workers) 
start_index = range(start_game_index, end_game_index, games_per_worker)

# Make urls
base_url = "https://api.rawg.io/api/games/"
urls = [base_url + csv_data[i][0] for i in range(number_of_games)]

In [13]:
print(*start_index)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99


In [14]:
urls[:10]

['https://api.rawg.io/api/games/3498',
 'https://api.rawg.io/api/games/4200',
 'https://api.rawg.io/api/games/3328',
 'https://api.rawg.io/api/games/12020',
 'https://api.rawg.io/api/games/5679',
 'https://api.rawg.io/api/games/802',
 'https://api.rawg.io/api/games/13536',
 'https://api.rawg.io/api/games/3439',
 'https://api.rawg.io/api/games/4291',
 'https://api.rawg.io/api/games/4062']

In [15]:
# Time
t0 = time()

with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
    temp = functools.partial(threading, games_per_worker=games_per_worker, urls=urls)
    executor.map(temp, start_index)
    
    
# Time diff
print(time()-t0)

6.20949912071228
